In [2]:
import pandas as pd
import numpy as np
import tqdm
from tqdm._tqdm_notebook import tqdm_notebook

In [3]:
CRAWL_EMBEDDING_PATH = '../input/crawl-300d-2M.vec'
GLOVE_EMBEDDING_PATH = '../input/glove.840B.300d.txt'

In [31]:
train = pd.read_hdf('../input/train.h5')
test = pd.read_hdf('../input/test.h5')

In [4]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [5]:
train["severe_toxicity"].value_counts().sort_index()

0.000000    1700238
0.000455          1
0.000466          1
0.000468          1
0.000470          1
0.000473          1
0.000543          1
0.000546          1
0.000547          1
0.000549          1
0.000550          1
0.000555          1
0.000556          1
0.000558          1
0.000561          1
0.000563          1
0.000569          1
0.000570          1
0.000580          1
0.000591          1
0.000591          1
0.000593          1
0.000593          3
0.000594          1
0.000595          1
0.000596          2
0.000598          1
0.000599          1
0.000599          1
0.000600          1
             ...   
0.369626          1
0.370240          1
0.370968          1
0.380305          1
0.384940          1
0.385604          1
0.389124          1
0.391632          1
0.393048          1
0.398633          1
0.400000         52
0.412836          1
0.412903          1
0.420485          1
0.434527          1
0.439405          1
0.452580          1
0.463168          1
0.466238          1


In [8]:
train[train["severe_toxicity"] == 1]["comment_text"].iloc[0]

'Awesome! Lets cut the head off hate! Lets stab it and burn its house down! lets march together and beat hate with rocks and clubs and run out out of town! Hooray for us for showing hate we will not tolerate it and will kill or beat down any that oppose or stance against hate. \n\nYeah, that sounds like the good guys. Why is it so hard for people to admit that using violence to achieve a goal is not acceptable in a civil society?'

In [5]:
train.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,...,2006,rejected,0,0,0,1,0,0.0,4,47


In [6]:
train.columns

Index(['id', 'target', 'comment_text', 'severe_toxicity', 'obscene',
       'identity_attack', 'insult', 'threat', 'asian', 'atheist', 'bisexual',
       'black', 'buddhist', 'christian', 'female', 'heterosexual', 'hindu',
       'homosexual_gay_or_lesbian', 'intellectual_or_learning_disability',
       'jewish', 'latino', 'male', 'muslim', 'other_disability',
       'other_gender', 'other_race_or_ethnicity', 'other_religion',
       'other_sexual_orientation', 'physical_disability',
       'psychiatric_or_mental_illness', 'transgender', 'white', 'created_date',
       'publication_id', 'parent_id', 'article_id', 'rating', 'funny', 'wow',
       'sad', 'likes', 'disagree', 'sexual_explicit',
       'identity_annotator_count', 'toxicity_annotator_count'],
      dtype='object')

In [7]:
train[train["severe_toxicity"] > 0.5]["comment_text"]

44188                  Kill all the nigggers and pedophiles!
1341396    BURN THIS DOG RAPING WHITE NIGER ALIVE\nTIME f...
1370031    The righteous ALT LEFT will eradicate fascist ...
1377740    Awesome! Lets cut the head off hate! Lets stab...
1443046    .\n.\nIs there really a God ?\n... I once thou...
1519955    We will send wave after wave of American soldi...
1532869    You are murdering feckless SCUM.  Your mother ...
1679044    DOGS----\nEAT\nSHIT\nSLEEP\nBARK\n\nNIGERS\nEA...
Name: comment_text, dtype: object

In [8]:
text = train["comment_text"]

In [9]:
text[text == "Take care not to believe your own bullshit, see On Bullshit."]

Series([], Name: comment_text, dtype: object)

In [10]:
ttext = test["comment_text"]

In [11]:
ttext[ttext == "Take care not to believe your own bullshit, see On Bullshit."]

Series([], Name: comment_text, dtype: object)

In [12]:
# do the embeddings have text in other languages?
text.head()

0    This is so cool. It's like, 'would you want yo...
1    Thank you!! This would make my life a lot less...
2    This is such an urgent design problem; kudos t...
3    Is this something I'll be able to install on m...
4                 haha you guys are a bunch of losers.
Name: comment_text, dtype: object

In [18]:
train[train["atheist"] == 1]["comment_text"].iloc[1]

"I don't get your shallow take on Atheism, then.  Who should we read if not Christopher Hitchens? \nWhat book on theism or atheism would you recommend, since you knocked a few of my favorite writers? Or is the existence of non-existence of a sky god off limits for discussion?  \n\nBTW- as a devout atheist I consider an agnostic to be an intellectual slacker.  That's okay- to slack off- but don't write about an issue that you care so little about.  \n\nThanks your your reading recommendations re: shedding light on religion in America."

In [19]:
train[train["atheist"] == 1].iloc[1]

id                                                                                242326
target                                                                               0.2
comment_text                           I don't get your shallow take on Atheism, then...
severe_toxicity                                                                        0
obscene                                                                                0
identity_attack                                                                      0.3
insult                                                                               0.1
threat                                                                                 0
asian                                                                                  0
atheist                                                                                1
bisexual                                                                               0
black                

In [21]:
len(train[train["muslim"] == 1])

12690

In [78]:
hi = train[(train["muslim"] == 1) & train["comment_text"].map(lambda x: "muslim" in x.lower() or "islam" in x.lower() or "moslem" in x.lower() or "muslums" in x.lower())]

In [79]:
len(hi)

12684

In [86]:
 bye = train[(train["muslim"] == 1) & train["comment_text"].map(lambda x: "muslim" not in x.lower() and "islam" not in x.lower() and "moslem" not in x.lower() and "muslums" not in x.lower())]
    
    

In [87]:
for x in bye["comment_text"]:
    print(x)
    print("--------------")

OK, I know who you are now.  And you repeat your same tired little slogan.  Perhaps you find it "clever".  But you still haven't answered my post.  And you haven't read al-Qur'an and are therefore unqualified to 'instruct' us concerning its contents.
--------------
Even their moderate clerics agree the Koran is riddled with archaic scripture calling for beheadings which doesn't leave much room for any misinterpretation.

I might even be on someone's list for not referring to it as the Quran.
--------------
Nope - read the documents more carefully.  The divine truth is revealed in many ways and many paths.  Yes, that means beyond the confines of the Roman Catholic Church.  For example, do you think 500-million Buddhists or 1.8 billion Muslins are wrong?  Of course not!  The deep abiding presence of the incardinate God is on oneness and nonduality​.
--------------
The Taliban is the same terrorist organization that entered a peaceful community opposed to their governance and hanged the 8

In [75]:
len(bye)

25

In [90]:
train[train["comment_text"] == "Nope - read the documents more carefully.  The divine truth is revealed in many ways and many paths.  Yes, that means beyond the confines of the Roman Catholic Church.  For example, do you think 500-million Buddhists or 1.8 billion Muslins are wrong?  Of course not!  The deep abiding presence of the incardinate God is on oneness and nonduality​."].iloc[0]

id                                                                               6042581
target                                                                                 0
comment_text                           Nope - read the documents more carefully.  The...
severe_toxicity                                                                        0
obscene                                                                                0
identity_attack                                                                        0
insult                                                                                 0
threat                                                                                 0
asian                                                                                  0
atheist                                                                                0
bisexual                                                                               0
black                